In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
import pandas as pd 
%matplotlib inline

In [2]:
img=cv2.imread('../images/cr7.jpg')
cv2.imshow('frame',img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
mp_pose=mp.solutions.pose.Pose(static_image_mode=True,
                               model_complexity=2,
                               enable_segmentation=True,
                               
                            )

In [ ]:
results=mp_pose.process(frame)

In [ ]:
pose_landmarks=results.pose_landmarks.landmark
pose_features=np.array([[pose_landmark.x,pose_landmark.y,pose_landmark.z,pose_landmark.visibility] for pose_landmark in pose_landmarks])

In [ ]:
pose_features.shape

In [ ]:
results.pose_landmarks is None

In [ ]:
np.array([[0.0,0.0,0.0] for i in range(33)])

In [ ]:
pose_df=pd.DataFrame(data=pose_features,columns=['x','y','z','Vis'])

In [ ]:
pose_df[['x','y']]

In [ ]:
class Pose:

    def __init__(self,static_image_mode=False,trackconf=0.5,detectionconf=0.5,
                 smooth_landmarks=True,model_complexity=2):
        
        self.static_image_mode=static_image_mode
        self.model_complexity=model_complexity
        self.trackconf=trackconf
        self.detectionconf=detectionconf
        self.smooth_landmarks=smooth_landmarks
        self.mp_pose=mp.solutions.pose
        self.mp_drawing=mp.solutions.drawing_utils
        self.mp_drawing_styles=mp.solutions.drawing_styles
        
        
        ## calling the object 

        self.pose=self.mp_pose.Pose(model_complexity=self.model_complexity,static_image_mode=self.static_image_mode,
                                    smooth_landmarks=self.smooth_landmarks,
                                    min_detection_confidence=self.detectionconf,
                                    min_tracking_confidence=self.trackconf)
        
        

    def get_pose_features(self,img):

        results=self.pose.process(img)

        

        self.lmks=results.pose_landmarks
        print('Hello')
        
        if self.lmks is None:
            print('None ')
            self.pose_features=np.array([[0.0,0.0,0.0] for i in range(33)])

        else:
            print('Hello')

            self.pose_features=np.array([[landmark.x,landmark.y,landmark.z]for landmark in self.lmks.landmark])


    def draw_landmarks(self,img):

       

       self.get_pose_features(img) 
       self. mp_drawing.draw_landmarks(
        img,
        self.lmks,
        self.mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=self.mp_drawing_styles.get_default_pose_landmarks_style())
    
   
    def find_angle(self,img,joints,draw=True):

        self.get_pose_features(img=img)
        h,w,c=img.shape
        j1,j2,j3=joints

        x1,y1=self.pose_features[j1][:2]
        x2,y2=self.pose_features[j2][:2]
        x3,y3=self.pose_features[j3][:2]
        x1,x2,x3=int(x1*w),int(x2*w),int(x3*w)
        y1,y2,y3=int(y1*h),int(y2*h),int(y3*h)
        # print(x1,y1)
        radian=np.arctan2(y3-y2,x3-x2)-np.arctan2(y1-y2,x1-x2)

        degree=(180/np.pi)*radian

        if draw:

            cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 3)
            cv2.line(img, (x3, y3), (x2, y2), (255, 255, 255), 3)
            cv2.circle(img, (x1, y1), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x1, y1), 15, (0, 0, 255), 2)
            cv2.circle(img, (x2, y2), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), 15, (0, 0, 255), 2)
            cv2.circle(img, (x3, y3), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x3, y3), 15, (0, 0, 255), 2)
            cv2.putText(img, str(int(degree)), (x2 - 50, y2 + 50),
                        cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
        

        return degree


    def get_bbox(self,img,draw=True):
        h,w,c=img.shape
        self.get_pose_features(img)
        print(self.pose_features)
        x_features=self.pose_features[:,0]
        y_features=self.pose_features[:,1]  
        x_min,x_max=int(np.min(x_features)*w),int(np.max(x_features)*w)
        
        y_min,y_max=int(np.min(y_features)*h),int(np.max(y_features)*h)

        if draw:
            img_copy=img.copy()
            cv2.rectangle(img_copy,(x_min,y_min),(x_max,y_max),(255,0,0),thickness=2)
            
            return img_copy,[(x_min,y_min),(x_max,y_max)]
        
            
        else:
            return [(x_min,y_min),(x_max,y_max)]
        

        
        

In [5]:
from Pose import Pose 

In [6]:
img=cv2.imread('../images/cr7.jpg')

In [7]:
p=Pose()

In [10]:
f=p.get_pose_features(img)

In [11]:
f

: 

In [ ]:
p.draw_landmarks(img)

In [ ]:
bbox_img,_=p.get_bbox(img)

In [ ]:
import yaml

In [ ]:
yaml_path='joint_index.yaml'

with open(yaml_path,'r') as file:
    joint_config=yaml.safe_load(file)

In [ ]:
joint_config

In [ ]:
joint_config['Hello']

In [19]:
cv2.imshow('frame',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'imshow'
> Overload resolution failed:
>  - mat is not a numpy array, neither a scalar
>  - Expected Ptr<cv::cuda::GpuMat> for argument 'mat'
>  - Expected Ptr<cv::UMat> for argument 'mat'


In [ ]:
features=p.pose_features

In [29]:
from Pose import Pose
import cv2
import numpy as np
import winsound

In [30]:
img=cv2.imread('../images/cr7.jpg')

In [31]:
p=Pose()

In [34]:
bbox=p.get_bbox(img,draw=False)

[[ 0.61208344  0.2070476  -0.04316162]
 [ 0.61683065  0.18803257 -0.0673872 ]
 [ 0.62018174  0.18680792 -0.06766601]
 [ 0.62326729  0.18560839 -0.06761133]
 [ 0.61300325  0.18701126 -0.0312877 ]
 [ 0.61354816  0.18556607 -0.0316847 ]
 [ 0.6137408   0.18404821 -0.03181257]
 [ 0.64145964  0.18112336 -0.112813  ]
 [ 0.62957472  0.18226489  0.04870193]
 [ 0.62460494  0.2210529  -0.05239495]
 [ 0.62005389  0.2208382  -0.007198  ]
 [ 0.70426327  0.27115282 -0.15843786]
 [ 0.63978076  0.27229628  0.1440215 ]
 [ 0.79000634  0.33595994 -0.25640282]
 [ 0.64036721  0.42373636  0.18855384]
 [ 0.88377875  0.38603866 -0.33726889]
 [ 0.63075852  0.55102009  0.1554841 ]
 [ 0.90946901  0.39479911 -0.36227617]
 [ 0.62407583  0.57728577  0.15577415]
 [ 0.91324043  0.39224422 -0.37714273]
 [ 0.61773521  0.57325763  0.12171237]
 [ 0.90256786  0.39320567 -0.34392449]
 [ 0.61895245  0.56534612  0.1373504 ]
 [ 0.64194196  0.58593094 -0.0984684 ]
 [ 0.62948471  0.56272471  0.0981843 ]
 [ 0.52449399  0.72705817

In [35]:
p1,p2=bbox

In [38]:
def fall_detection(p1,p2):
    x_min,y_min=p1
    x_max,y_max=p2
    h=y_max-y_min
    w=x_max-x_min

    if w>h:
        return True
    else:
        return False

In [39]:
if fall_detection(p1,p1):
    print('Fall Detected')

else:
    print('No Fall ')

No Fall 


In [22]:
def draw_border(img, point1, point2,color=(0,255,0)):

    x1, y1 = point1
    x3, y3 = point2
    x2,y2=x1,y3
    x4, y4 = x3,y1 
    h=int(y3-y1)

    line_length=h//6  

   

    cv2.line(img, (x1, y1), (x1 , y1 + line_length), color, 4)  #-- top-left
    cv2.line(img, (x1, y1), (x1 + line_length , y1), color, 4)

    cv2.line(img, (x2, y2), (x2 , y2 - line_length), color, 4)  #-- bottom-left
    cv2.line(img, (x2, y2), (x2 + line_length , y2), color, 4)

    cv2.line(img, (x3, y3), (x3 - line_length, y3), color, 4)  #-- top-right
    cv2.line(img, (x3, y3), (x3, y3 - line_length), color, 4)

    cv2.line(img, (x4, y4), (x4 , y4 + line_length), color, 4)  #-- bottom-right
    cv2.line(img, (x4, y4), (x4 - line_length , y4), color, 4)

    return img


In [23]:
img=cv2.imread('../images/cr7.jpg')

In [24]:
draw_border(img,p1,p2,color=(0,0,255))

array([[[ 83,  73,  79],
        [ 82,  72,  78],
        [ 81,  72,  75],
        ...,
        [130,  97,  88],
        [132,  99,  90],
        [133, 100,  91]],

       [[ 82,  72,  78],
        [ 82,  72,  78],
        [ 81,  72,  75],
        ...,
        [131,  98,  89],
        [131,  98,  89],
        [132,  99,  90]],

       [[ 81,  72,  75],
        [ 81,  72,  75],
        [ 81,  72,  75],
        ...,
        [131,  98,  89],
        [131,  98,  89],
        [131,  98,  89]],

       ...,

       [[ 92, 194, 136],
        [ 92, 194, 136],
        [ 91, 193, 135],
        ...,
        [ 99, 193, 139],
        [100, 194, 140],
        [100, 194, 140]],

       [[ 90, 192, 134],
        [ 90, 192, 134],
        [ 90, 192, 134],
        ...,
        [ 98, 192, 138],
        [ 99, 193, 139],
        [ 98, 192, 138]],

       [[ 89, 191, 133],
        [ 89, 191, 133],
        [ 89, 191, 133],
        ...,
        [ 97, 191, 137],
        [ 98, 192, 138],
        [ 98, 192, 138]]

In [25]:
cv2.imshow('',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
cap=cv2.VideoCapture('../videos/fall.mp4')

while cap.isOpened():

    ret,frame=cap.read()
    

    if not ret:
        break
    
    frame_copy,(corn1,corn2)=p.get_bbox(frame,draw=True)
    x1,y1=corn1
    x2,y2=corn2

    height=y2-y1
    width=x2-x1

    if width > height:
        h,w,c=frame.shape

        warning_frame=np.zeros((h,w,c),dtype=np.uint8)
        warning_frame[:, :] = (0, 0, 255)
        fall_frame=cv2.addWeighted(frame_copy,1,warning_frame,0.7,0)
        cv2.putText(fall_frame, 'FALL', (x1, y2 -height),
                        cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
        cv2.imshow('',fall_frame)
        frequency = 2500  # Adjust the frequency as needed
        duration = 500  # Adjust the duration as needed
        winsound.Beep(frequency, duration)
    else:
    
        cv2.imshow('',frame_copy)

    if cv2.waitKey(1)==27:
        break


cv2.destroyAllWindows()
    

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [

In [ ]:
x1,y1=corn1
x2,y2=corn2

height=y2-y1
width=x2-x1

In [ ]:
height


In [ ]:
width

In [ ]:
class test:

    def tester(self):
        self.counter=0
        self.counter+=1

In [ ]:
t=test()

In [ ]:
t.tester()

In [ ]:
t.tester()

In [ ]:
t.counter